In [1]:
import requests
import re
import json
from datetime import datetime
import pandas as pd


In [2]:
def get_data_from_macrotrends(cityName, cityCode, df, area_miles):
    url = "https://www.macrotrends.net/production/global_metrics/city_state_pop_data.php?citystate="+str(cityCode)+"&slug=population&chart=chart1"

    # Add browser headers
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/115.0.0.0 Safari/537.36","Referer": "https://www.macrotrends.net/"
    }

    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    html = resp.text
    data = json.loads(html)
    city, state = cityName.split(',')
    val_2020 = next((value for ts, value in data["data"] if ts == 1577836800000), None)
    val_2021 = next((value for ts, value in data["data"] if ts == 1609459200000), None)
    val_2022 = next((value for ts, value in data["data"] if ts == 1640995200000), None)
    val_2023 = next((value for ts, value in data["data"] if ts == 1672531200000), None)
    val_2024 = next((value for ts, value in data["data"] if ts == 1704067200000), None)
    
    df.loc[len(df)] = {
        'city': city, 'state': state, 'area_miles': area_miles, 'year': '2020', 'population' : val_2020,
        'density': round(val_2020/area_miles, 2), 'city_state' : city+", "+state
    }
    df.loc[len(df)] = {
        'city': city, 'state': state, 'area_miles': area_miles, 'year': '2021', 'population' : val_2021,
        'density': round(val_2021/area_miles, 2), 'city_state' : city+", "+state
    } 
    df.loc[len(df)] = {
        'city': city, 'state': state, 'area_miles': area_miles, 'year': '2022', 'population' : val_2022,
        'density': round(val_2022/area_miles, 2), 'city_state' : city+", "+state
    } 
    df.loc[len(df)] = {
        'city': city, 'state': state, 'area_miles': area_miles, 'year': '2023', 'population' : val_2023,
        'density': round(val_2023/area_miles, 2), 'city_state' : city+", "+state
    } 
    df.loc[len(df)] = {
        'city': city, 'state': state, 'area_miles': area_miles, 'year': '2024', 'population' : val_2024,
        'density': round(val_2024/area_miles, 2), 'city_state' : city+", "+state
    } 




In [3]:
cities_df = pd.read_csv('../data/uscities.csv')
cities_df

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18832416,10943.7,shape,False,True,America/New_York,1,11229 11228 11226 11225 11224 11222 11221 1122...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11885717,3165.7,shape,False,True,America/Los_Angeles,1,91367 90291 90293 90292 91316 91311 90035 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8489066,4590.3,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6113982,4791.1,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,6046392,1386.2,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31249,Mount Wilson,Mount Wilson,NV,Nevada,32017,Lincoln,38.2350,-114.4504,0,2.2,shape,False,True,America/Los_Angeles,5,89043,1840039843
31250,Kohatk,Kohatk,AZ,Arizona,4021,Pinal,32.5781,-112.0032,0,77.3,shape,False,False,America/Phoenix,3,85634,1840022983
31251,Ironville,Ironville,PA,Pennsylvania,42013,Blair,40.6586,-78.2155,0,59.2,shape,False,False,America/New_York,3,16686,1840152922
31252,Newkirk,Newkirk,NM,New Mexico,35019,Guadalupe,35.0635,-104.2715,0,0.0,shape,False,False,America/Denver,3,88417,1840024978


In [4]:
def get_area_in_miles(cityState, cities_df):
    city, state = cityState.split(',')

    filtered_df = cities_df[
        cities_df['city'].str.contains(city, case=False, na=False) &
        cities_df['state_id'].str.contains(state, case=False, na=False)
    ]

    if filtered_df.empty:
        print(f"No match found for city: {city}, state: {state}")
        return None  # or some default value
        
    return round((filtered_df['population'] / filtered_df['density']).values[0], 2)
   

In [5]:
city_ids = {'Atlanta,GA':22922,
'Birmingham,AL':22936,
'Charleston,SC':22952,
'Chattanooga,TN':22955,
'Cleveland,MS':22959,
'Columbia,SC':22962,
'Huntsville,AL':23016,
'Knoxville,TN':23032,
'Little Rock,AR':23049 ,
'Louisville,KY':23053,
'Memphis,TN':23063,
'Nashville,TN':23077,
'Myrtle Beach,SC':23455,
'Richmond,VA':23115,
'Columbia,SC':22962,
'Savannah,GA':23137,
'Mobile,AL':23069,
'Kansas City,MO':23028,
'Springfield,MO':23150,
'Jackson,MS':23019}

df = pd.DataFrame()
df = pd.DataFrame(columns=['city','state', 'area_miles', 'year', 'population', 'density', 'city_state'])

for city, city_id in city_ids.items():
    area_miles = get_area_in_miles(city, cities_df)
    get_data_from_macrotrends(city, city_id, df, area_miles)
    
    
df

,city,state,area_miles,year,population,density,city_state
0,Atlanta,GA,3657.47,2020,5803000,1586.62,"Atlanta, GA"
1,Atlanta,GA,3657.47,2021,5911000,1616.14,"Atlanta, GA"
2,Atlanta,GA,3657.47,2022,6013000,1644.03,"Atlanta, GA"
3,Atlanta,GA,3657.47,2023,6106000,1669.46,"Atlanta, GA"
4,Atlanta,GA,3657.47,2024,6193000,1693.25,"Atlanta, GA"
...,...,...,...,...,...,...,...
90,Jackson,MS,639.64,2020,420000,656.62,"Jackson, MS"
91,Jackson,MS,639.64,2021,426000,666.00,"Jackson, MS"
92,Jackson,MS,639.64,2022,431000,673.82,"Jackson, MS"
93,Jackson,MS,639.64,2023,436000,681.63,"Jackson, MS"


In [6]:
df.to_excel('../data/Census.xlsx', index=False)